In [1]:
%matplotlib inline

# APSG tutorial - structural geology module for Python - Part 2

In [2]:
from apsg import *
apsg_conf['figsize'] = (9, 7)

## Matrix like classes and tensors
**APSG** provides matrix-like classes to work with tensor quantities used commonly in structural geology analysis. It includes `DefGrad3` and `VelGrad3` for deformation and velocity gradient, `Stress3` for stress tensor, `Ellipsoid` for quadratic forms and `Ortensor3` for orientation tensor. All these classes support common matrix mathematical operations and provide basic methods and properties.

All matrix-like objects could be created either by passing nested list or tuple or providing individual components to class method `from_comp`

In [3]:
F = defgrad([[2, 0, 1], [0, 1, 0], [0, 0, 0.5]])
F

DeformationGradient3
[[2.  0.  1. ]
 [0.  1.  0. ]
 [0.  0.  0.5]]

In [4]:
F = defgrad.from_comp(xx=2, zz=0.5, xz=1)
F

DeformationGradient3
[[2.  0.  1. ]
 [0.  1.  0. ]
 [0.  0.  0.5]]

For multiplifications of matrix or vectors we have to use matmul `@` operator

In [5]:
v = vec3('z') # unit-length vector in direction af axis z
u = F @ v
u

Vector3(1, 0, 0.5)

`I` property returns inverse matrix

In [6]:
F.I @ u

Vector3(0, 0, 1)

To transpose matrix, we can use `T` property and for multiplification we have to use `@` operator

In [7]:
F.T @ F

DeformationGradient3
[[4.   0.   2.  ]
 [0.   1.   0.  ]
 [2.   0.   1.25]]

In [8]:
v @ F.T @ F @ v

1.25

Eigenvalues and eigenvectors could be obtained by methods `eigenvalues` and `eigenvectors`. Individual eigenvalues and eigen vectors could be accessed by properties `E1`, `E2`, `E3` and `V1`, `V2`, `V3`

#### Deformation gradient and rotations

Deformation gradient `DeformationGradient3` could describe distorsion, dilation and rigid-body rotation. All **APSG** features provides `transform` method which transform then using provided deformation gradient.

The rigid-body rotation could be either extracted from deformation gradient using `R` method:

In [9]:
R = F.R
R

DeformationGradient3
[[ 0.928  0.     0.371]
 [ 0.     1.     0.   ]
 [-0.371  0.     0.928]]

or could be created of one of the class methods like `from_axisangle`, defining axis of rotation and angle

In [10]:
R = defgrad.from_axisangle(lin(120, 50), 60)
R

DeformationGradient3
[[ 0.552 -0.753  0.359]
 [ 0.574  0.655  0.492]
 [-0.605 -0.065  0.793]]

`from_two_vectors`, where axis of rotation is perpendicular to both vectors and angle is angle of vectors

In [11]:
R = defgrad.from_two_vectors(lin(120, 50), lin(270, 80))
R

DeformationGradient3
[[ 0.938  0.074  0.339]
 [ 0.186  0.718 -0.671]
 [-0.294  0.692  0.66 ]]

In [12]:
lin(120, 50).transform(R)

L:270/80

or using `from_two_pairs` method, to describe rotation between two coordinate systems. Note that pair define X axis as lineation vector and Z axis as foliation vector.

In [13]:
p1 = pair(150, 60, 90, 40)
p2 = pair(45, 30, 10, 25)
R = defgrad.from_two_pairs(p1, p2)
R

DeformationGradient3
[[-0.071  0.97   0.234]
 [-0.874 -0.174  0.453]
 [ 0.48  -0.173  0.86 ]]

In [14]:
p1.transform(R)

P:45/30-10/25

### Ellipsoids

In deformation analysis, the quadratic forms are represented by `Ellipsoid` class. It could be used to represents either ellipsoid objects or finite strain ellipsoid.

It provides additional methods and properties including `lambda1`, `lambda2` and `lambda3` for square-root of eigenvalues, Woodcock's `shape` and `strength`, `k`, `K`, `d` and `D` for Flinn's and Ramsay symmetries and intensities, `lode` for Lode's parameter etc. For more check documentation. Eigenvectors could be also represented by linear or planar features using properties `eigenlins` and `eigenfols`.

We can create `Ellipsoid` object similarly to `Matrix3` (note that only components of upper triangular part are available in `from_comp` method due to matrix symmetry), or you can use aditional class methods `from_defgrad` and `from_stretch`.

In [15]:
B = ellipsoid.from_defgrad(F)  # Finger deformation tensor
B

Ellipsoid
[[5.   0.   0.5 ]
 [0.   1.   0.  ]
 [0.5  0.   0.25]]
(λ1:2.25, λ2:1, λ3:0.445)

In above example, the Finger deformation tensor `B` represents finite strain ellipsoid reulting from deformation described by deformation gradient `F`. We can explore several parameters:

In [16]:
print(f'Principal stretches: Sx={B.lambda1}, Sy={B.lambda2}, Sz={B.lambda3}')
print(f'Principal strain ratios: Rxy={B.Rxy}, Ryz={B.Ryz}')
print(f"Flinn's finite strain parameters: d={B.d}, k={B.k}")
print(f"Ramsay's finite strain parameters: d={B.D}, k={B.K}")
print(f"Woodcock's parameters: strength={B.strength}, shape={B.shape}")
print(f"Watterson's strain intesity: s{B.r}")
print(f"Nadai's natural octahedral unit shear: {B.goct}")
print(f"Nadai's natural octahedral unit strain: {B.eoct}")
print(f"Lode's parameter: {B.lode}")

Principal stretches: Sx=2.247679020649623, Sy=1.0, Sz=0.4449033829176287
Principal strain ratios: Rxy=2.247679020649623, Ryz=2.247679020649623
Flinn's finite strain parameters: d=1.764484592491078, k=1.0
Ramsay's finite strain parameters: d=1.3118699860194967, k=1.0
Woodcock's parameters: strength=1.6197962748564998, shape=1.0
Watterson's strain intesity: s3.495358041299246
Nadai's natural octahedral unit shear: 1.3225581202197991
Nadai's natural octahedral unit strain: 1.1453689300917396
Lode's parameter: 0.0


In [17]:
C = ellipsoid.from_defgrad(F, 'right')  # Green's deformation tensor
C

Ellipsoid
[[4.   0.   2.  ]
 [0.   1.   0.  ]
 [2.   0.   1.25]]
(λ1:2.25, λ2:1, λ3:0.445)

In [18]:
v @ C @ v

1.25

### Ortensor class
`Ortensor3` class represents orientation tensor of set of vectors, linear or planar features. In adition to `Ellipsoid` methods and properties, it provides properties to describe orientation distribution, e.g. Vollmer's `P`, `G`, `R` and `B` indexes, `Intensity` for Lisle intensity index and `MAD` for approximate angular deviation.

In [19]:
l = linset.random_fisher(position=lin(120,40))
ot = l.ortensor()
# or
ot = ortensor.from_features(l)
ot

OrientationTensor3
[[ 0.158 -0.207 -0.186]
 [-0.207  0.46   0.383]
 [-0.186  0.383  0.382]]
(λ1:0.953, λ2:0.235, λ3:0.19)

In [20]:
ot.eigenvalues()

(0.9086798155777692, 0.055075989269152054, 0.03624419515307836)

In [21]:
ot.eigenvectors()

(Vector3(-0.348, 0.696, 0.628),
 Vector3(-0.936, -0.214, -0.28),
 Vector3(-0.061, -0.685, 0.726))

In [22]:
ot.kind

'LLS'

The instances of `Stress3`, `Ellipsoid` and `Ortensor3` also provides `eigenlins` and `eigenfols` properties to represent principal axes and planes

In [23]:
ot.eigenlins

(L:117/39, L:13/16, L:265/47)

In [24]:
ot.eigenfols

(S:297/51, S:193/74, S:85/43)

In [25]:
ot.strength, ot.shape

(1.6108567796345399, 6.69944312795964)

In [26]:
ot.k, ot.d

(13.157218408138034, 3.070684605230345)

In [27]:
ot.K, ot.D

(6.69944312795964, 2.008365085670506)

In [28]:
ot.P, ot.G, ot.R

(0.8536038263086172, 0.03766358823214738, 0.10873258545923509)

In [29]:
ot.MAD

17.589325255926394